<a href="https://colab.research.google.com/github/dannnn30/DSS/blob/main/OR_TOOLS_pt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 23.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0


#1. Constraint Programming (CP)
##Penugasan Kerja dengan Constraint Programming

In [ ]:
from ortools.sat.python import cp_model

# Data
num_projects = 5
num_employees = 4
weeks = 10

# Durasi proyek dalam minggu
project_durations = [4, 5, 6, 3, 4]

# Jumlah pekerjaan per proyek (total jam kerja)
project_workload = [40, 50, 60, 30, 45]

# Batas akhir proyek dalam minggu
project_deadlines = [6, 7, 10, 5, 8]

# Kapasitas kerja tiap karyawan (jam/minggu)
employee_capacity = [15, 10, 20, 12]

# Inisialisasi model
model = cp_model.CpModel()

# Variabel: x[e][p] adalah jumlah jam kerja karyawan e di proyek p
x = {}
for e in range(num_employees):
    for p in range(num_projects):
        x[e, p] = model.NewIntVar(0, employee_capacity[e] * weeks, f'x_{e}_{p}')

# 1. Setiap proyek harus memiliki minimal 2 karyawan yang mengerjakannya
for p in range(num_projects):
    assigned = []
    for e in range(num_employees):
        is_assigned = model.NewBoolVar(f'is_assigned_{e}_{p}')
        model.Add(x[e, p] > 0).OnlyEnforceIf(is_assigned)
        model.Add(x[e, p] == 0).OnlyEnforceIf(is_assigned.Not())
        assigned.append(is_assigned)

    model.Add(sum(assigned) >= 2)


# 2. Total jam kerja per proyek harus memenuhi kebutuhan proyek
for p in range(num_projects):
    model.Add(sum(x[e, p] for e in range(num_employees)) == project_workload[p])

# 3. Batasan kapasitas kerja tiap karyawan per minggu
for e in range(num_employees):
    model.Add(sum(x[e, p] for p in range(num_projects)) <= employee_capacity[e] * weeks)

# 4. Total jam kerja per proyek tidak boleh melebihi deadline proyek
for p in range(num_projects):
    model.Add(sum(x[e, p] for e in range(num_employees)) <= project_deadlines[p] * sum(employee_capacity))

# 5. Meminimalkan total jam kerja keseluruhan
model.Minimize(sum(x[e, p] for e in range(num_employees) for p in range(num_projects)))

# Solver
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Output hasil
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Solusi Optimal Ditemukan:")
    for e in range(num_employees):
        for p in range(num_projects):
            if solver.Value(x[e, p]) > 0:
                print(f'Karyawan {e+1} bekerja di Proyek {p+1} selama {solver.Value(x[e, p])} jam')
else:
    print("Tidak ada solusi yang memenuhi semua kendala.")


Solusi Optimal Ditemukan:
Karyawan 1 bekerja di Proyek 1 selama 39 jam
Karyawan 2 bekerja di Proyek 1 selama 1 jam
Karyawan 2 bekerja di Proyek 2 selama 1 jam
Karyawan 3 bekerja di Proyek 3 selama 1 jam
Karyawan 3 bekerja di Proyek 4 selama 29 jam
Karyawan 3 bekerja di Proyek 5 selama 44 jam
Karyawan 4 bekerja di Proyek 2 selama 49 jam
Karyawan 4 bekerja di Proyek 3 selama 59 jam
Karyawan 4 bekerja di Proyek 4 selama 1 jam
Karyawan 4 bekerja di Proyek 5 selama 1 jam


#2. Bin Packing dan Knapsack Problems
##Seleksi Barang dalam Game RPG

In [ ]:
!pip install ortools --upgrade  # Ensure OR-Tools is upgraded to latest
from ortools.linear_solver import pywraplp # Correct import


def solve_rpg_knapsack():
    # Data Problem
    values = [9, 8, 5, 4, 7, 6, 8, 7]  # Nilai kegunaan masing-masing item
    weights = [[5, 7, 1, 1, 4, 2, 3, 1]]  # Berat masing-masing item
    volumes = [[10, 12, 2, 2, 6, 3, 5, 1]]  # Volume masing-masing item
    capacities = [15, 30]  # Kapasitas: [berat, volume]
    num_items = len(values)

    # Solver OR-Tools
    solver = pywraplp.Solver.CreateSolver('SCIP') # Initialize solver using pywraplp

    # Define variables
    x = [solver.IntVar(0, 1, f'x_{i}') for i in range(num_items)] # 1 if item is selected, 0 otherwise

    # Constraints
    # Weight constraint
    solver.Add(sum(x[i] * weights[0][i] for i in range(num_items)) <= capacities[0])
    # Volume constraint
    solver.Add(sum(x[i] * volumes[0][i] for i in range(num_items)) <= capacities[1])

    # Objective function: Maximize total value
    solver.Objective().SetMaximization()
    for i in range(num_items):
        solver.Objective().SetCoefficient(x[i], values[i])


    # Menjalankan solver
    status = solver.Solve()

    # Menampilkan hasil
    if status == pywraplp.Solver.OPTIMAL:
        print("Total nilai kegunaan maksimal:", solver.Objective().Value())
        print("Barang yang dipilih:")
        total_weight, total_volume = 0, 0
        for i in range(num_items):
            if x[i].solution_value() > 0:
                print(f"- Item ke-{i+1}: Berat = {weights[0][i]} kg, Volume = {volumes[0][i]} liter, Nilai = {values[i]}")
                total_weight += weights[0][i]
                total_volume += volumes[0][i]

        print(f"Total Berat: {total_weight} kg, Total Volume: {total_volume} liter")
    else:
        print("The problem does not have an optimal solution.")

# Menjalankan solver
solve_rpg_knapsack()


Total nilai kegunaan maksimal: 40.00000000000001
Barang yang dipilih:
- Item ke-1: Berat = 5 kg, Volume = 10 liter, Nilai = 9
- Item ke-3: Berat = 1 kg, Volume = 2 liter, Nilai = 5
- Item ke-4: Berat = 1 kg, Volume = 2 liter, Nilai = 4
- Item ke-5: Berat = 4 kg, Volume = 6 liter, Nilai = 7
- Item ke-7: Berat = 3 kg, Volume = 5 liter, Nilai = 8
- Item ke-8: Berat = 1 kg, Volume = 1 liter, Nilai = 7
Total Berat: 15 kg, Total Volume: 26 liter


#3. Constraint Programming (CP)
##Penjadwalan Produksi Film

In [ ]:
from ortools.sat.python import cp_model

def schedule_film_shooting():
    # Data
    days = range(1, 6)  # Hari 1 sampai 5
    scenes = range(6)  # 6 Adegan
    actors = {
        0: {1, 2, 3, 5},  # Aldo
        1: {1, 2, 4, 5},  # Bella
        2: {1, 3, 4, 5},  # Charlie
        3: {2, 3, 4, 5}   # Dina
    }
    locations = {
        0: {1, 3, 5},  # Istana
        1: {2, 4, 5},  # Hutan
        2: {1, 2, 3}   # Pasar
    }
    scene_requirements = [
        (0, [0, 2]),    # Adegan 1: Istana (Aldo, Charlie)
        (1, [1, 3]),    # Adegan 2: Hutan (Bella, Dina)
        (2, [0, 1, 2]), # Adegan 3: Pasar (Aldo, Bella, Charlie)
        (1, [2, 3]),    # Adegan 4: Hutan (Charlie, Dina)
        (0, [0, 2, 3]), # Adegan 5: Istana (Aldo, Charlie, Dina)
        (2, [1, 3])     # Adegan 6: Pasar (Bella, Dina)
    ]

    # Model CP-SAT
    model = cp_model.CpModel()

    # Variabel: Hari syuting untuk setiap adegan
    scene_days = {s: model.NewIntVar(1, 5, f'scene_{s}_day') for s in scenes}

    # Constraint: Setiap adegan harus dilakukan pada hari yang memenuhi syarat lokasi
    for s, (location, required_actors) in enumerate(scene_requirements):
        model.AddAllowedAssignments([scene_days[s]], [[d] for d in locations[location]])

        # Constraint: Setiap aktor harus tersedia pada hari itu
        for actor in required_actors:
            model.AddAllowedAssignments([scene_days[s]], [[d] for d in actors[actor]])

    # Solver
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # Hasil
    if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
        for s in scenes:
            print(f'Adegan {s+1} dijadwalkan pada hari {solver.Value(scene_days[s])}')
    else:
        print("Tidak ada solusi yang memenuhi semua constraint.")

# Jalankan solver
schedule_film_shooting()

Adegan 1 dijadwalkan pada hari 1
Adegan 2 dijadwalkan pada hari 2
Adegan 3 dijadwalkan pada hari 1
Adegan 4 dijadwalkan pada hari 4
Adegan 5 dijadwalkan pada hari 3
Adegan 6 dijadwalkan pada hari 2


#4. Network Flow Problems
##Internet Packet Routing



In [ ]:
from ortools.linear_solver import pywraplp

def internet_packet_routing():
    # Inisialisasi model jaringan aliran minimum biaya
    # Use pywraplp.Solver instead of SimpleMinCostFlow
    solver = pywraplp.Solver.CreateSolver('SCIP')

    # Node pada jaringan (A = 0, B = 1, C = 2, D = 3, E = 4, F = 5, G = 6)
    start_nodes = [0, 0, 1, 1, 2, 2, 3, 3, 4, 5, 4, 5]
    end_nodes = [1, 2, 3, 4, 3, 5, 4, 6, 6, 6, 5, 4]
    capacities = [20, 15, 25, 10, 20, 30, 15, 10, 25, 20, 10, 15]  # Bandwidth maksimum (Mbps)
    unit_costs = [2, 3, 1, 4, 2, 2, 3, 5, 1, 2, 3, 2]  # Biaya per unit data
    num_arcs = len(start_nodes)


    # Define flow variables for each arc
    flow = {}
    for i in range(num_arcs):
        flow[i] = solver.IntVar(0, capacities[i], f'flow_{i}')

    # Constraints: Flow conservation at each node (except source and sink)
    for node in range(1, 6):  # Nodes B to F
        incoming_flow = sum(flow[i] for i in range(num_arcs) if end_nodes[i] == node)
        outgoing_flow = sum(flow[i] for i in range(num_arcs) if start_nodes[i] == node)
        solver.Add(incoming_flow == outgoing_flow)


    # Constraint: Total flow from source (A) equals supply
    solver.Add(sum(flow[i] for i in range(num_arcs) if start_nodes[i] == 0) == 30)

    # Objective function: Minimize total cost
    objective = solver.Objective()
    for i in range(num_arcs):
        objective.SetCoefficient(flow[i], unit_costs[i])
    objective.SetMinimization()

    # Menjalankan solver
    status = solver.Solve()

    # Menampilkan hasil
    if status == pywraplp.Solver.OPTIMAL:
        print("Solusi optimal ditemukan!")
        print("Jalur data yang dipilih:")
        total_cost = 0
        for i in range(num_arcs):
            if flow[i].solution_value() > 0:
                print(f"Server {start_nodes[i]} -> Server {end_nodes[i]}: {flow[i].solution_value()} Mbps")
                total_cost += flow[i].solution_value() * unit_costs[i]
        print(f"Total biaya pengiriman: {total_cost}")
    else:
        print("Tidak ada solusi optimal yang ditemukan.")

# Jalankan solver
internet_packet_routing()

Solusi optimal ditemukan!
Jalur data yang dipilih:
Server 0 -> Server 1: 15.0 Mbps
Server 0 -> Server 2: 15.0 Mbps
Server 1 -> Server 3: 15.0 Mbps
Server 2 -> Server 5: 15.0 Mbps
Server 3 -> Server 4: 15.0 Mbps
Server 4 -> Server 6: 15.0 Mbps
Server 5 -> Server 6: 15.0 Mbps
Total biaya pengiriman: 210.0


#5. Bin Packing dan Knapsack Problems
##Penempatan Kontainer di Kapal Kargo

In [ ]:
from ortools.linear_solver import pywraplp

# Data kontainer (berat, luas, dan nilai prioritas)
weights = [10, 8, 12, 7, 5, 15, 6, 9, 11, 4]  # Berat (ton)
volumes = [20, 15, 25, 12, 10, 30, 14, 18, 22, 8]  # Luas (m²)
values = [60, 50, 80, 40, 30, 90, 35, 55, 70, 20]  # Nilai Prioritas

# Kapasitas maksimal kapal
max_weight = 50  # ton
max_volume = 100  # m²

# Inisialisasi solver menggunakan pywraplp
solver = pywraplp.Solver.CreateSolver('SCIP')

# Define variables
x = {}
for i in range(len(weights)):
    x[i] = solver.IntVar(0, 1, f'x_{i}')

# Constraints
# Weight constraint
solver.Add(sum(x[i] * weights[i] for i in range(len(weights))) <= max_weight)
# Volume constraint
solver.Add(sum(x[i] * volumes[i] for i in range(len(weights))) <= max_volume)

# Objective function: Maximize total value
objective = solver.Objective()
for i in range(len(weights)):
    objective.SetCoefficient(x[i], values[i])
objective.SetMaximization()

# Menjalankan solver
status = solver.Solve()

# Menampilkan hasil
packed_items = []
total_weight = 0
total_volume = 0
total_value = 0

if status == pywraplp.Solver.OPTIMAL:
    for i in range(len(weights)):
        if x[i].solution_value() > 0:
            packed_items.append(f"C{i+1}")  # Menyimpan ID kontainer
            total_weight += weights[i]
            total_volume += volumes[i]
            total_value += values[i]

    print("\nKontainer yang dimuat ke kapal:", packed_items)
    print(f"Total Berat: {total_weight} ton dari kapasitas {max_weight} ton")
    print(f"Total Luas: {total_volume} m² dari kapasitas {max_volume} m²")
    print(f"Total Nilai Prioritas: {total_value}")
else:
    print("The problem does not have an optimal solution.")


Kontainer yang dimuat ke kapal: ['C1', 'C2', 'C3', 'C8', 'C9']
Total Berat: 50 ton dari kapasitas 50 ton
Total Luas: 100 m² dari kapasitas 100 m²
Total Nilai Prioritas: 315
